## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline

from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

In [2]:
submission = pd.read_csv('./data/submission.csv')
with open('./pickle/data.npy', 'rb') as f:
    X_trains= np.load(f)
    y_trains = np.load(f)
    TEST = np.load(f)

In [7]:
RX_train = pickle.load(open("./pickle/RX_train.pkl", 'rb'))
RX_test = pickle.load(open("./pickle/RX_test.pkl", 'rb'))

RS_train = pickle.load(open("./pickle/RS_train.pkl", 'rb'))
RS_test = pickle.load(open("./pickle/RS_test.pkl", 'rb'))

RR_train = pickle.load(open("./pickle/RR_train.pkl", 'rb'))
RR_test = pickle.load(open("./pickle/RR_test.pkl", 'rb'))

In [8]:
RX_test

array([[8],
       [8],
       [8],
       ...,
       [6],
       [8],
       [6]])

In [9]:
RS_test

array([[6],
       [8],
       [8],
       ...,
       [6],
       [8],
       [0]])

In [10]:
RR_test

array([[6],
       [9],
       [8],
       ...,
       [6],
       [3],
       [0]])

In [11]:
new_X_train = np.concatenate((RX_train, RS_train, DS_train), axis = 1)
new_X_test = np.concatenate((RX_test, RS_test,  DS_test), axis = 1)
print('origin :', X_trains.shape, TEST.shape)
print('new : ', new_X_train.shape, new_X_test.shape)

origin : (2048, 28, 28, 3) (20480, 28, 28, 3)
new :  (2048, 3) (20480, 3)


In [12]:
new_y_train = np.argmax(y_trains, axis=1).reshape((y_trains.shape[0], 1))

In [13]:
lgbm = LGBMClassifier(n_estimators=200)
lgbm.fit(new_X_train, new_y_train)
final_pred = lgbm.predict(new_X_test)
final_pred

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([6, 8, 8, ..., 6, 8, 4])

In [20]:
submission['digit'] = DS_test
submission

,id,digit
0,2049,6
1,2050,9
2,2051,8
3,2052,0
4,2053,3
...,...,...
20475,22524,4
20476,22525,1
20477,22526,6
20478,22527,3


In [21]:
submission.to_csv('./submission/ReXEnsemble.csv', index=False)